# Strands Agents with AWS Neptune MCP Server Demo

This notebook demonstrates how to use [Strands Agents](https://strandsagents.com/) with the [AWS Neptune MCP Server](https://awslabs.github.io/mcp/servers/amazon-neptune-mcp-server/) to intelligently interact with Amazon Neptune graphs.

## Overview

This demo showcases:
1. **Neptune MCP Server Setup**: Connect to Neptune Analytics or Neptune Database
2. **Graph Status Monitoring**: Check if the graph is available
3. **Schema Discovery**: Retrieve and analyze graph structure
4. **Intelligent Querying**: Use Strands Agents to generate and execute openCypher queries
5. **Autonomous Graph Analysis**: Let the agent explore and analyze graph data

## Architecture

```
User Prompt → Strands Agent → Neptune MCP Server → Neptune Graph
                    ↓
              Amazon Bedrock
              (Claude 3.5 Sonnet)
```

In [30]:
import os
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp import stdio_client, StdioServerParameters
import boto3

## 2. Configuration

In [ ]:
# Configuration
# For Neptune Analytics, use format: neptune-graph://<graph-identifier>
# For Neptune Database, use format: neptune-db://<cluster-endpoint>

NEPTUNE_ENDPOINT = "neptune-graph://<INPUT_GRAPH_ID>"
AWS_REGION = "us-west-2"
AWS_PROFILE = None

# Bedrock configuration
BEDROCK_MODEL = "us.anthropic.claude-3-5-sonnet-20240620-v1:0"
BEDROCK_REGION = "us-west-2"

# Set environment variables for Neptune MCP server
os.environ['NEPTUNE_ENDPOINT'] = NEPTUNE_ENDPOINT
os.environ['AWS_REGION'] = AWS_REGION
if AWS_PROFILE:
    os.environ['AWS_PROFILE'] = AWS_PROFILE

print(f"Configuration set:")
print(f"  Neptune Endpoint: {NEPTUNE_ENDPOINT}")
print(f"  AWS Region: {AWS_REGION}")
print(f"  Bedrock Model: {BEDROCK_MODEL}")

## 3. Initialize Neptune MCP Client

The Neptune MCP server provides three main tools:
- `get_status()`: Check if the graph is available
- `get_schema()`: Retrieve the graph schema
- `run_query()`: Execute openCypher or Gremlin queries

In [32]:
# Initialize Neptune MCP Client
def create_neptune_mcp_client():
    """Create and return a Neptune MCP client."""
    return MCPClient(lambda: stdio_client(
        StdioServerParameters(
            command="uvx",
            args=["awslabs.amazon-neptune-mcp-server@latest"],
            env={
                "NEPTUNE_ENDPOINT": NEPTUNE_ENDPOINT,
                "AWS_REGION": AWS_REGION,
                "FASTMCP_LOG_LEVEL": "INFO"
            }
        )
    ))

print("Neptune MCP client creator ready")

Neptune MCP client creator ready


## 4. Direct Neptune MCP Operations

Let's test basic Neptune operations using the MCP server directly.
reference: https://strandsagents.com/latest/documentation/docs/api-reference/tools/

In [33]:
# Test Neptune MCP server directly
print("Testing Neptune MCP Server...\n")

with create_neptune_mcp_client() as neptune_mcp:
    # List available tools
    tools = neptune_mcp.list_tools_sync()
    print(f"Available Neptune MCP Tools: {len(tools)}")

    for tool in tools:
        print(f"  - {tool.tool_name}: {tool.tool_spec}")
    print()

Testing Neptune MCP Server...

Available Neptune MCP Tools: 4
  - get_graph_status: {'inputSchema': {'json': {'properties': {}, 'title': 'get_statusArguments', 'type': 'object'}}, 'name': 'get_graph_status', 'description': 'Get the status of the currently configured Amazon Neptune graph.', 'outputSchema': {'json': {'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'get_statusOutput', 'type': 'object'}}}
  - get_graph_schema: {'inputSchema': {'json': {'properties': {}, 'title': 'get_schemaArguments', 'type': 'object'}}, 'name': 'get_graph_schema', 'description': 'Get the schema for the graph including the vertex and edge labels as well as the\n    (vertex)-[edge]->(vertex) combinations.\n    ', 'outputSchema': {'json': {'$defs': {'Node': {'description': 'Defines a node type in the graph schema.\n\nNodes represent entities in the graph database and can have labels\nand properties that describe their characteristics.\n\nAttributes:\n    labe

## 5. Create Strands Agent with Neptune MCP Tools

Now let's create an intelligent agent that can interact with Neptune autonomously.

In [34]:

NEPTUNE_AGENT_SYSTEM_PROMPT = """
You are an expert Neptune graph database analyst and assistant.

Your capabilities include:
1. Checking the status and availability of Neptune graphs
2. Analyzing graph schemas to understand data structure
3. Generating and executing openCypher queries to retrieve information
4. Providing insights and analysis about graph data

When analyzing graphs:
- Always start by checking the graph status and schema
- Generate efficient openCypher queries based on the schema
- Explain your findings clearly with relevant statistics
- Provide actionable insights from the data

When writing openCypher queries:
- Use proper syntax for Neptune Analytics/Database
- Include LIMIT clauses to avoid overwhelming results
- Handle potential errors gracefully
- Explain what each query does before executing it
"""

print("Agent system prompt defined")

Agent system prompt defined


In [35]:
# Create the Neptune Graph Analysis Agent
def create_neptune_agent():
    """Create a Strands Agent with Neptune MCP tools."""
    neptune_mcp = create_neptune_mcp_client()
    
    with neptune_mcp:
        # Get available Neptune tools
        tools = neptune_mcp.list_tools_sync()
        
        # Create the agent
        agent = Agent(
            system_prompt=NEPTUNE_AGENT_SYSTEM_PROMPT,
            tools=tools,
        )
        
        return agent, neptune_mcp

print("Neptune agent creator ready")

Neptune agent creator ready


## 6. Agent Demonstrations

Let's see the agent in action with various tasks.

### 6.1 Basic Graph Health Check

In [36]:
print("=" * 80)
print("TASK 1: Graph Health Check")
print("=" * 80)

agent, neptune_mcp = create_neptune_agent()

with neptune_mcp:
    prompt = """
    Check the health and status of the Neptune graph. 
    Provide a summary of its availability and readiness.
    """
    
    print(f"\nUser Prompt: {prompt.strip()}\n")
    print("Agent Response:")
    print("-" * 80)
    
    response = agent(prompt)
    print(response)
    print("\n" + "=" * 80 + "\n")

TASK 1: Graph Health Check

User Prompt: Check the health and status of the Neptune graph. 
    Provide a summary of its availability and readiness.

Agent Response:
--------------------------------------------------------------------------------
I'll check the health and status of the Neptune graph for you.
Tool #1: get_graph_status
## Neptune Graph Status Summary

✅ **Graph Status: Available**

The Neptune graph is currently **healthy and ready for use**. Here's what this means:

- **Availability**: The graph database is online and accessible
- **Readiness**: Ready to accept and process queries
- **Operations**: You can safely perform read and write operations
- **Performance**: The system is operating normally

The "Available" status indicates that:
- All database services are running properly
- The graph is ready to handle openCypher and Gremlin queries
- No maintenance or backup operations are currently blocking access
- The system has sufficient resources to process requests

You

### 6.2 Schema Analysis and Exploration

In [37]:
print("=" * 80)
print("TASK 2: Schema Analysis")
print("=" * 80)

agent, neptune_mcp = create_neptune_agent()

with neptune_mcp:
    prompt = """
    Analyze the Neptune graph schema and provide:
    1. A summary of the node types (labels) present
    2. A summary of the relationship types present
    3. Key properties for important node and relationship types
    4. An overview of the graph structure and domain it represents
    """
    
    print(f"\nUser Prompt: {prompt.strip()}\n")
    print("Agent Response:")
    print("-" * 80)
    
    response = agent(prompt)
    print(response)
    print("\n" + "=" * 80 + "\n")

TASK 2: Schema Analysis

User Prompt: Analyze the Neptune graph schema and provide:
    1. A summary of the node types (labels) present
    2. A summary of the relationship types present
    3. Key properties for important node and relationship types
    4. An overview of the graph structure and domain it represents

Agent Response:
--------------------------------------------------------------------------------
I'll help you analyze the Neptune graph schema. Let me start by checking the graph status and then examining the schema structure.
Tool #1: get_graph_status
Great! The graph is available. Now let me examine the schema to understand the structure of the graph.
Tool #2: get_graph_schema
Now let me query some sample data to better understand the graph structure and get more detailed information about the properties and data patterns.
Tool #3: run_opencypher_query

Tool #4: run_opencypher_query
Let me examine some sample nodes to better understand the properties and data structure:

### 6.3 Data Exploration and Statistics

In [38]:
print("=" * 80)
print("TASK 3: Data Exploration and Statistics")
print("=" * 80)

agent, neptune_mcp = create_neptune_agent()

with neptune_mcp:
    prompt = """
    Explore the Neptune graph and provide statistics:
    1. Total count of nodes and relationships
    2. Count of nodes by type/label
    3. Count of relationships by type
    4. Any interesting patterns or insights you notice
    
    Please execute appropriate openCypher queries to gather this information.
    """
    
    print(f"\nUser Prompt: {prompt.strip()}\n")
    print("Agent Response:")
    print("-" * 80)
    
    response = agent(prompt)
    print(response)
    print("\n" + "=" * 80 + "\n")

TASK 3: Data Exploration and Statistics

User Prompt: Explore the Neptune graph and provide statistics:
    1. Total count of nodes and relationships
    2. Count of nodes by type/label
    3. Count of relationships by type
    4. Any interesting patterns or insights you notice

    Please execute appropriate openCypher queries to gather this information.

Agent Response:
--------------------------------------------------------------------------------
I'll help you explore the Neptune graph and provide comprehensive statistics. Let me start by checking the graph status and schema to understand the data structure.
Tool #1: get_graph_status
Great! The graph is available. Now let me examine the schema to understand the structure of nodes and relationships.
Tool #2: get_graph_schema
Perfect! Now I understand the graph structure. This appears to be a knowledge graph with:
- **Nodes**: Entity, Chunk, DocumentId
- **Relationships**: FROM (Chunk→DocumentId), CONTAINS (Chunk→Entity)

Let me now

### 6.4 Natural Language Query Execution

In [39]:
print("=" * 80)
print("TASK 4: Natural Language Query")
print("=" * 80)

agent, neptune_mcp = create_neptune_agent()

with neptune_mcp:
    prompt = """
    Find the most connected nodes in the graph. 
    Show me the top 5 nodes with the most relationships,
    including their labels and the count of connections.
    """
    
    print(f"\nUser Prompt: {prompt.strip()}\n")
    print("Agent Response:")
    print("-" * 80)
    
    response = agent(prompt)
    print(response)
    print("\n" + "=" * 80 + "\n")

TASK 4: Natural Language Query

User Prompt: Find the most connected nodes in the graph. 
    Show me the top 5 nodes with the most relationships,
    including their labels and the count of connections.

Agent Response:
--------------------------------------------------------------------------------
I'll help you find the most connected nodes in the graph. Let me start by checking the graph status and schema to understand the data structure.
Tool #1: get_graph_status
Great! The graph is available. Now let me examine the schema to understand the structure of nodes and relationships.
Tool #2: get_graph_schema
Perfect! Now I understand the graph structure. The graph contains:
- **Entity** nodes
- **Chunk** nodes (with text and metadata properties)
- **DocumentId** nodes
- **FROM** relationships (Chunk → DocumentId)
- **CONTAINS** relationships (Chunk → Entity)

Now I'll write an openCypher query to find the top 5 most connected nodes by counting both incoming and outgoing relationships.


### 6.5 Complex Graph Analysis

In [40]:
# Task 5: Complex analysis
print("=" * 80)
print("TASK 5: Complex Graph Analysis")
print("=" * 80)

agent, neptune_mcp = create_neptune_agent()

with neptune_mcp:
    # This prompt should be customized based on your graph domain
    prompt = """
    Perform a comprehensive analysis of the graph:
    1. Identify the most important node types based on connectivity
    2. Find any obvious patterns in the relationship structure
    3. Look for potential data quality issues (disconnected nodes, missing properties, etc.)
    4. Provide recommendations for graph queries that would be useful for this data
    
    Be thorough and execute multiple queries as needed to gather insights.
    """
    
    print(f"\nUser Prompt: {prompt.strip()}\n")
    print("Agent Response:")
    print("-" * 80)
    
    response = agent(prompt)
    print(response)
    print("\n" + "=" * 80 + "\n")

TASK 5: Complex Graph Analysis

User Prompt: Perform a comprehensive analysis of the graph:
    1. Identify the most important node types based on connectivity
    2. Find any obvious patterns in the relationship structure
    3. Look for potential data quality issues (disconnected nodes, missing properties, etc.)
    4. Provide recommendations for graph queries that would be useful for this data

    Be thorough and execute multiple queries as needed to gather insights.

Agent Response:
--------------------------------------------------------------------------------
I'll perform a comprehensive analysis of your Neptune graph. Let me start by checking the graph status and understanding its structure.
Tool #1: get_graph_status
Great! The graph is available. Now let me examine the schema to understand the data structure:
Tool #2: get_graph_schema
Excellent! Now I have a clear understanding of the graph structure. This appears to be a knowledge graph with document chunks, entities, and do